In [374]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from MiningMinima import *

# Developing subroutine to extract function derivative evaluations

In [375]:
# use mm object to get stuff 
seq1, seq2 = 'gg', 'cc'
mm_sim = MiningMinima(seq1,seq2)

In [376]:
mm_sim.multifunc.dfunc(theta, dE_dtheta)

In [377]:
print dE_dtheta

vector1_double[0.0789342, -0.105804, 0.884873, 0.557242, 2.73597, 0.376245, 0.573104, 0.711411, -2.00864, 2.52009, 1.04075, 2.19686, 1.91306, 0.225052]


In [378]:
# get pose and scorefxn 
pose = protocols.recces.pose_setup_turner(seq1=seq1,seq2=seq2)
scorefxn = core.scoring.ScoreFunctionFactory.create_score_function('stepwise/rna/turner_new')

# set up movemap -- for now just get from the MM object -- and min map
movemap = MoveMap()
movemap.assign(mm_sim.movemap)  
min_map = core.optimization.MinimizerMap()
min_map.setup(pose,movemap)

# set up min options
min_options = core.optimization.MinimizerOptions(
    'lbfgs_armijo_nonmonotone',1e-15,True,False,False)

# get scorefunction ready for minimization
start_score = scorefxn(pose)
pose.energies().set_use_nblist(pose,min_map.domain_map(),True)
multifunc = core.optimization.AtomTreeMultifunc(pose,min_map,scorefxn)
scorefxn.setup_for_minimizing(pose,min_map) # this is the key command
scorefxn.setup_for_derivatives(pose)

min_dofs = Vector1([0.0]*min_map.nangles())
min_map.copy_dofs_from_pose(pose,min_dofs)
start_func_val = multifunc(min_dofs)
theta = Vector1(list(min_dofs))
dE_dtheta = Vector1(list(min_dofs))
multifunc.dfunc(theta,dE_dtheta)

# set up minimizer and run
minimizer = core.optimization.Minimizer(multifunc,min_options)
for _ in range(10): minimizer.run(min_dofs)
print dE_dtheta

core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 4.76837e-07 Deriv= -7.07846e-05 Finite Diff= 0.000269696
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 1.90735e-06 Deriv= -0.000478678 Finite Diff= 0.000156473
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 7.45058e-09 Deriv= -7.94523e-07 Finite Diff= 0.029088
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate

In [362]:
multifunc.dfunc(min_dofs, dE_dtheta)

In [363]:
dE_dtheta

vector1_double[0.000256151, -0.000524317, 5.53975e-05, -0.000277307, 0.000177263, -0.000191802, -3.91371e-05, -0.000107799, -0.00015844, 2.08362e-05, -0.000179215, -0.000100009, 8.31817e-05, 0.000467006]

# Developing subroutine to calculate hessian from derivative evaluations

In [379]:
def hessian_at_min(min_dofs,multifunc,h=1e-6):
    min_dofs = np.array(min_dofs) # take advantage of numpy array indexing
    n_dofs = len(min_dofs)
    hessian = np.zeros((n_dofs,n_dofs))
    plus = Vector1([0.0]*n_dofs)
    minus = Vector1(list(plus))
    
    for ii in range(n_dofs):
        new_dofs = min_dofs[:]
        new_dofs[ii] += h*180./np.pi
        multifunc.dfunc(Vector1(list(new_dofs)),plus)
        new_dofs[ii] -= 2.*h*180/np.pi
        multifunc.dfunc(Vector1(list(new_dofs)),minus)
        row = (np.array(plus) - np.array(minus))/2./h
        hessian[ii] = row*(np.pi/180)**1 # need to figure out conversion
    return hessian

In [380]:
for _ in range(10):
    minimizer.run(min_dofs)

core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.8147e-08 Deriv= -7.94523e-07 Finite Diff= 0.00619367
core.optimization.LineMinimizer: [ ERROR ] Inaccurate G!

In [381]:
hessian = hessian_at_min(min_dofs,multifunc)

In [382]:
from scipy.linalg import eigh

In [383]:
[eigenvalues, modes] = eigh(hessian)

In [384]:
pseudo_det = 1.
for eigen in eigenvalues:
    if eigen > 0:
        pseudo_det *= eigen
    

In [385]:
multifunc(min_dofs) + 0.5*np.log(pseudo_det) - 0.5*min_map.nangles()*np.log(2*np.pi)

-81.24878712827854

# Developing subroutine to mode scan using AtomTreeMultifunc

In [386]:
def mode_scan(min_dofs, multifunc, mode, limit=np.pi/3, dx=0.005):
    min_dofs = np.array(min_dofs)
    delta_array = np.linspace(-limit, limit, int(2*limit/dx))
    
    result = np.zeros_like(delta_array)
    
    for ii, delta in enumerate(delta_array):
        new_dofs = min_dofs[:] + delta*mode*180/np.pi
        result[ii] = multifunc(Vector1(list(new_dofs)))
    
    return result    
    
    

In [393]:
total_log_partition = 0 
dx = 0.005
for mode in modes.T: # columns of array are eigenvectors
    result = mode_scan(min_dofs, multifunc, mode, limit=4*np.pi/3, dx=dx)
    result -= multifunc(min_dofs)
    total_log_partition += np.log(np.trapz(np.exp(-result),dx=dx))
    

In [394]:
multifunc(min_dofs) - total_log_partition

-20.168257390802886

In [ ]:
sims = ['aaa_uuu', 'aa_uu', 'cc_gg', 'ccc_ggg', 'cc_gg', 'gaa_uuc', 'gac_guc', 'gag_cuc', 'gau_auc', 'ga_uc', 'gcc_ggc', 'gcg_cgc', 'gga_ucc', 'gg_cc', 'ggc_gcc', 'ggg_ccc', 'ggu_acc', 'gu_ac', 'guc_gac', 'guu_aac']

harmonic_E = []
RECCES_E = []
DOS_E = []

E = np.linspace(-100,800,9001)

for sim in sims:
    
    seqs = sim.split('_')
    seq1 = seqs[0]
    seq2 = seqs[1]
    
    fuck = MiningMinima(seq1, seq2)
    
    hist = SingleHistSimulation('./../../../recces_rosetta/results/'+sim+'/ST/')
    
    harmonic_E.append(fuck.harmonic_free_energy)
    RECCES_E.append(hist.value)
    DOS_E.append(-scipy.special.logsumexp(-E, b=fuck.dos(E)))

In [ ]:
print RECCES_E
print harmonic_E
print DOS_E
plt.scatter(RECCES_E, DOS_E)
plt.plot([-45,-15],[-45,-15])

In [ ]:
from recces.util import *
from recces.data import *

In [ ]:
sim = SingleHistSimulation('./../../../recces_rosetta/results/'+seq1+'_'+seq2+'/ST/')

In [ ]:
print sim.value
-np.log(np.trapz(sim._dos, dx=0.1))
'guu_acc'.split('_')

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy

E = np.linspace(-100,100,2001)

plt.plot(E, fuck.dos(E)*np.exp(-E))
plt.plot(sim._dos_scores[:2000], sim._dos[:2000]*np.exp(-sim._dos_scores[:2000]))
plt.xlabel('energy (REU)')
plt.ylabel('$g(E)e^{-E/kT}$')
plt.legend(('MM', 'RECCES'))
print -scipy.special.logsumexp(-E, b=fuck.dos(E)), sim.value, fuck.harmonic_free_energy

In [ ]:
print fuck.anharmonic_free_energy
print fuck.harmonic_free_energy

In [ ]:

print fuck.modes[:,0]
pmm.apply(fuck.min_pose)

In [ ]:
n_struct = 200

ensemble = fuck.harmonic_ensemble(n_struct = n_struct)
#print ensemble[2,:]-min_dofs
#print np.sqrt(H)/np.sqrt((2*np.pi)**fuck.n_dofs)*np.exp(-np.dot(fuck.eigenvalues, ((min_dofs-ensemble[0,:])*np.pi/180)**2)/2)

In [ ]:
pose = Pose()
pose.assign(fuck.min_pose)

hist_min = -100
hist_max = 100
delE = 0.1

hist_scores = np.linspace(hist_min, hist_max, int((hist_max-hist_min)/delE)+1)

hist = np.zeros_like(hist_scores)

for i in range(n_struct):
    
    dofs = ensemble[i,:]
    
    for key in fuck.dof_dict: pose.set_torsion(fuck.dof_dict[key], dofs[key])
        
    energy = fuck.scorefxn(pose)
    
    if energy < hist_max+delE:
             
        hist_idx = math.floor((energy-hist_min)/delE)
        hist_idx = int(hist_idx)
        hist[hist_idx] += 1


In [ ]:
shit = hist*np.exp(hist_scores)/np.sum(hist)
shit = shit/np.sum(shit)
print -scipy.special.logsumexp(-hist_scores, b = shit)
plt.plot(hist_scores, shit*np.exp(-hist_scores)/torsion_volume(seq1, seq2))
plt.plot(sim._dos_scores[:2000], sim._dos[:2000]*np.exp(-sim._dos_scores[:2000]))

In [ ]:
pose = Pose()
pose_new = Pose()
pose.assign(fuck.min_pose)
for key in fuck.dof_dict: pose.set_torsion(fuck.dof_dict[key],ensemble[0,:][key])

pose_new.assign(pose)

E_0 = fuck.scorefxn(fuck.min_pose)
energy = E_0

min_dofs = [fuck.min_pose.torsion(fuck.dof_dict[key]) for key in fuck.dof_dict]
curr_dofs = min_dofs
print min_dofs

Z = 0
H = np.product(fuck.eigenvalues)

for j in range(1,n_struct):
    
    trial_move = ensemble[j, :]
     
    for key in fuck.dof_dict: pose_new.set_torsion(fuck.dof_dict[key], trial_move[key]) 

    energy_new = fuck.scorefxn(pose_new)
    
    if np.exp(-(energy_new-energy)) > np.random.random():
        
        energy = energy_new
        pose.assign(pose_new)
        curr_dofs = trial_move
        
    

        #p = np.sqrt(H/(2*np.pi)**fuck.n_dofs)*np.exp(-np.dot(fuck.eigenvalues, ((trial_move-min_dofs)*(np.pi/180)**2)**2)/2)
    if energy < hist_max+delE:
             
        hist_idx = math.floor((energy-hist_min)/delE)
        hist_idx = int(hist_idx)
        hist[hist_idx] += 1

    

In [ ]:
#plt.plot(hist_scores, hist)
shit = hist/np.sum(hist)
shit = shit*np.exp(hist_scores)
shit = shit/np.sum(shit)
#plt.plot(hist_scores, shit)
plt.plot(hist_scores, hist/np.sum(hist))
print -np.log(np.exp(-E_0)*(0.25)**fuck.n_dofs/0.35)


In [ ]:
print ensemble
pmm = PyMOLMover()
pmm.keep_history(True)
print ensemble[1,:]

In [ ]:
pose = Pose()
pose.assign(fuck.min_pose)

for i in range(200):
    
    dofs = ensemble[i,:]
    for key in fuck.dof_dict: pose.set_torsion(fuck.dof_dict[key], dofs[key]) 
    pmm.apply(pose)
    

In [ ]:
print np.cov(ensemble)

In [ ]:
w,v = np.linalg.eigh(np.cov(ensemble.T))
print fuck.eigenvalues

In [ ]:
print 1/w*180*180/np.pi/np.pi

In [ ]:

print fuck.min_energy - 0.5*fuck.n_dofs*np.log(2*np.pi) + 0.5*np.log(np.prod(1/w))

In [ ]:
print fuck.anharmonic_free_energy

In [ ]:
print fuck.min_pose

In [ ]:
mu = np.zeros(fuck.n_dofs)

In [ ]:
print mu

In [ ]:
mu = [fuck.min_pose.torsion(fuck.dof_dict[key]) for key in fuck.dof_dict]

In [ ]:
print mu
bitch = np.linalg.inv(fuck.hessian)

In [ ]:
shit = np.random.multivariate_normal(mu, bitch, size=(2000))

In [ ]:
print shit[0,:]

In [ ]:
random_struct = np.zeros((fuck.n_dofs, 2000))

for j in range(2000):
    
    random_struct[:,j] = np.random.multivariate_normal(mu, bitch)

In [ ]:
print random_struct

In [ ]:
print np.cov(random_struct)

In [ ]:
w,v = np.linalg.eigh(np.cov(random_struct))

In [ ]:
print 1/w


In [ ]:
cu_ag = MiningMinima('cuc', 'gag')

In [ ]:
cu_ag.harmonic_free_energy

In [ ]:
ensemble = cu_ag.harmonic_ensemble()

In [ ]:
cu_ag.eigenvalues

In [ ]:
np.diag(1/cu_ag.eigenvalues)

In [ ]:
np.matmul( np.matmul( cu_ag.modes, np.diag(1/cu_ag.eigenvalues)), cu_ag.modes.T)

In [ ]:
np.linalg.inv(cu_ag.hessian)